# Using LX-Translator to translate sentences
This is an example notebook that illustrates how you can use the LX-Translator web service to translate
Portuguese sentences to Chinese and back.

**Before you run this example**, replace `access_key_goes_here` by your webservice access key, below:

In [1]:
LXTRANSLATOR_WS_API_KEY = 'access_key_goes_here'
LXTRANSLATOR_WS_API_URL = 'https://portulanclarin.net/workbench/lx-translator/api/'

## Importing required Python modules
The next cell will take care of installing the `requests` package,
if not already installed, and make it available to use in this notebook.

In [2]:
try:
    import requests
except:
    !pip3 install requests
    import requests

## Wrapping the complexities of the JSON-RPC API in a simple, easy to use function

The `WSException` class defined below, will be used later to identify errors
from the webservice.

In [3]:
class WSException(Exception):
    'Webservice Exception'
    def __init__(self, errordata):
        "errordata is a dict returned by the webservice with details about the error"
        super().__init__(self)
        assert isinstance(errordata, dict)
        self.message = errordata["message"]
        # see https://json-rpc.readthedocs.io/en/latest/exceptions.html for more info
        # about JSON-RPC error codes
        if -32099 <= errordata["code"] <= -32000:  # Server Error
            if errordata["data"]["type"] == "WebServiceException":
                self.message += f": {errordata['data']['message']}"
            else:
                self.message += f": {errordata['data']!r}"
    def __str__(self):
        return self.message

The next function invoques the LX-Suite webservice through it's public JSON-RPC API.

In [4]:
def translate(text, source, target):
    '''
    Arguments
        text: a string with a maximum of 2000 characters, Portuguese or Chinese
              text, with the input to be translated
        source: is a string specifying the source language, which can be either
               'zh' or 'pt'
        target: is a string specifying the target language, which can be either
               'zh' or 'pt' and must be different than source.
    
    Returns a string with the translation in the target language.
    Raises a WSException if an error occurs.
    '''

    request_data = {
        'method': 'translate',
        'jsonrpc': '2.0',
        'id': 0,
        'params': {
            'text': text,
            'source': source,
            'target': target,
            'key': LXTRANSLATOR_WS_API_KEY,
        },
    }
    request = requests.post(LXTRANSLATOR_WS_API_URL, json=request_data)
    response_data = request.json()
    if "error" in response_data:
        raise WSException(response_data["error"])
    else:
        return response_data["result"]

Let us test the function we just defined:

In [5]:
text = '''Esta frase serve para testar o funcionamento do tradutor. Esta outra
frase faz o mesmo.'''

result = translate(text, source="pt", target="zh")
print(result)

这句话用于测试翻译的运作。 另一个
这句话也是一样。


## Getting the status of a webservice access key

In [6]:
def get_key_status():
    '''Returns a string with the detailed status of the webservice access key'''
    
    request_data = {
        'method': 'key_status',
        'jsonrpc': '2.0',
        'id': 0,
        'params': {
            'key': LXTRANSLATOR_WS_API_KEY,
        },
    }
    request = requests.post(LXTRANSLATOR_WS_API_URL, json=request_data)
    response_data = request.json()
    if "error" in response_data:
        raise WSException(response_data["error"])
    else:
        return response_data["result"]

In [7]:
get_key_status()

{'requests_remaining': 99999996,
 'chars_remaining': 999999710,
 'expiry': '2030-01-10T00:00+00:00'}